# Visualize Chromatin Configuration

Convert an x, y, z configuration file to PDB file format.

### Setup

In [1]:
# Built-in modules
import os
import sys

# Insert package root to system path
cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

In [2]:
# External modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Change working directory to package root
os.chdir(parent_dir)

### Load Configuration Output

In [16]:
#output_path = "/Users/jwakim/Desktop/manual_contact_map_processing/interact_4/representative_snapshot_sim16/Chr-1-73.csv"
output_path = "/Users/jwakim/Desktop/manual_contact_map_processing_batch_2/one_mark/Chr-1-199_4.csv"

In [17]:
poly_config = pd.read_csv(output_path, header=[0,1], index_col=0, sep=",")
poly_config.head()

r                              t3                            t2  \
            x          y          z         x         y         z         x   
0 -201.873677 -91.546333  33.460141 -0.120921  0.686930 -0.716593 -0.420971   
1 -201.950751 -76.254163  24.160288  0.348671  0.925060  0.150639  0.915217   
2 -191.664067 -61.688020  29.619473  0.453254  0.827571  0.331192  0.102654   
3 -182.766083 -47.829423  36.450174  0.686705  0.726913  0.005870 -0.001230   
4 -169.547771 -34.636504  33.186741  0.650407  0.643117 -0.404192  0.788688   

                      states chemical_mods         bead_length  
          y         z    HP1       H3K9me3 Unnamed: 12_level_1  
0  0.485197 -0.766399      2             0                16.5  
1  0.051489  0.399658      2             0                16.5  
2  0.992421  0.067545      2             0                16.5  
3  0.952408 -0.304823      2             0                16.5  
4  0.526309  0.317759      2             0                16.5

In [18]:
r_x = poly_config.loc[:, ("r", "x")].to_numpy()
r_y = poly_config.loc[:, ("r", "y")].to_numpy()
r_z = poly_config.loc[:, ("r", "z")].to_numpy()

HP1 = poly_config.loc[:, ("states", "HP1")].to_numpy()
#PRC1 = poly_config.loc[:, ("states", "PRC1")].to_numpy() * 10

# Express in PDB format

In [19]:
def get_PDB_string(x, y, z, ind, bound):
    """ Generate a line for a PDB  file corresponding to a single bead.

    Parameters
    ----------
    x : float
        x-position of the bead
    y : float
        y-position of the bead
    z : float
        z-position of the bead
    ind : int
        Index of the bead
    bound : int
        Indicates the binding state of the protein

    Returns
    -------
    str
        String representing the bead in PDB format
    """

    # Atom indicator (cols 1-4)
    str_out = "ATOM"

    # Atom Serial Number (cols 7-11)
    str_out += 2 * " "
    ind_out = f"{ind}"
    len_ind = len(ind_out)
    if len_ind > 5:
        raise ValueError("The size of the index is not supported in PDB format. Please enter an index between 1 and 99999.")
    pad_size = 5 - len_ind
    str_out += " " * pad_size + ind_out

    # Atom Name (cols 13-16)
    str_out += " "
    if bound == 0:
        str_out += "H   "
    elif bound == 1 or bound == 2:
        str_out += "O   "
    elif bound == 10 or bound == 20:
        str_out += "N   "
    elif bound == -1:
        str_out += "S   "
    else:
        str_out += "C   "

    # Alternate Location Indicator (col 17)
    str_out += " "

    # Residue Name (cols 18-20), Chain Identifier (col 22), Residue sequence number (cols 23-26)
    str_out += "MET A   1"

    # x-position (cols 31-38)
    str_out += "    "
    if x > 9999999 or x < -999999:
        raise ValueError("The size of the x-position is not supported in PDB format. Please limit to 7 characters.")
    if x >= 0:
        if x > 999999:
            str_x = '{: 8.3f}'.format(x)
        else:
            str_x = '{: 8.3f}'.format(x)
    else:
        if x < -99999:
            str_x = '{: 8.3f}'.format(x)
        else:
            str_x = '{: 8.3f}'.format(x)
    str_out += str_x

    # y-position (cols 39-46)
    # str_out += " "
    if y > 9999999 or y < -999999:
        raise ValueError("The size of the y-position is not supported in PDB format. Please limit to 7 characters.")
    if y >= 0:
        if y > 999999:
            str_y = '{: 8.3f}'.format(y)
        else:
            str_y = '{: 8.3f}'.format(y)
    else:
        if y < -99999:
            str_y = '{: 8.3f}'.format(y)
        else:
            str_y = '{: 8.3f}'.format(y)
    str_out += str_y

    # z-position (cols 47-54)
    # str_out += " "
    if z > 9999999 or z < -999999:
        raise ValueError("The size of the z-position is not supported in PDB format. Please limit to 7 characters.")
    if z >= 0:
        if z > 999999:
            str_z = '{: 8.3f}'.format(z)
        else:
            str_z = '{: 8.3f}'.format(z)
    else:
        if z < -99999:
            str_z = '{: 8.3f}'.format(z)
        else:
            str_z = '{: 8.3f}'.format(z)
    str_out += str_z
    return str_out

In [20]:
#pdb_path = "/Users/jwakim/Desktop/manual_contact_map_processing/interact_4/representative_snapshot_sim16/Chr-1-73.pdb"
pdb_path = "/Users/jwakim/Desktop/manual_contact_map_processing_batch_2/one_mark/Chr-1-199_4.pdb"
num_beads = len(r_x)

with open(pdb_path, 'w') as f:
    for i in range(num_beads):
        #line = get_PDB_string(r_x[i], r_y[i], r_z[i], i+1, HP1[i]+PRC1[i])
        line = get_PDB_string(r_x[i], r_y[i], r_z[i], i+1, HP1[i])
        f.write(line + "\n")

    # Draw reference points for circle enclosure
    line = get_PDB_string(900, 0, 0, num_beads+1, -1)
    f.write(line + "\n")
    line = get_PDB_string(0, 900, 0, num_beads+1, -1)
    f.write(line + "\n")
    line = get_PDB_string(-900, 0, 0, num_beads+1, -1)
    f.write(line + "\n")
    line = get_PDB_string(0, -900, 0, num_beads+1, -1)
    f.write(line + "\n")